<h1> Installing MySQL on OS X </h1>

Following this guide http://wpguru.co.uk/2015/11/how-to-install-mysql-on-mac-os-x-el-capitan/

<b><u>To install MySQL </b></u>: 

1) Download the DMG archive from http://dev.mysql.com/downloads/mysql/ <br />
2) Run the DMG file to install it on your computer <br />
3) Start MySQL by clicking apple -> system preferences -> MySQL -> Start MySQL Server
<br />
4) Altenatively run "sudo launchctl start com.mysql.mysql" from terminal

<b><u>To configure MySQL</b></u>: 
 
1) Open a terminal <br />
2) Add the following aliases: <br />

alias mysql=/usr/local/mysql/bin/mysql <br />
alias mysqladmin=/usr/local/mysql/bin/mysqladmin <br />

3) Alterntively, add 'export PATH="/usr/local/mysql/bin:$PATH"' to ./bash_profile <br />
4) Run the secure installation by typing mysql_secure_installation in terminal <br />

<b><u>To install the MySQL python module</b></u>: 

1) conda install pymysql

<h1> Import Python Modules </h1>

<h> Using the tutorial from http://zetcode.com/db/mysqlpython/ </h>

In [1]:
import pymysql as mdb
import sys

<h1> Connect to DB and get DB version</h1>

In [2]:
try:
    
    #Inputs to connect are MySQLServer, UserName, Password, Database
    con = mdb.connect('localhost', 'testuser', 'test623', 'testdb');

    #Execute commands by making a cursor, and sending .excute statements
    cur = con.cursor()
    cur.execute("SELECT VERSION()")

    #Fetch one line of the MySQL return using .fetchone()
    ver = cur.fetchone()

    print ("Database version : %s " % ver)
    
#Check for errors with .Error    
except mdb.Error as e:
    print("Error %d: %s" % (e.args[0],e.args[1]))
    sys.exit(1)
    
#Remeber to close the connection when finished    
finally:    
    if con:    
        con.close()

Database version : 5.7.13 


<h1> Create a new table called Writers and insert some data </h1>

Note, if you do not use with to clean up the connection you will need to use con.commit() and con.rollback() to deal with committing changes.  See the last section of http://zetcode.com/db/mysqlpython/ for details

In [3]:
con = mdb.connect('localhost', 'testuser', 'test623', 'testdb');

#With will close the connection after the code is done, 
#regardless of how the code exists. Use as an alternative to 'finally' statement
with con:
    
    cur = con.cursor()
    
    #Remove the table if it already exists
    cur.execute("DROP TABLE IF EXISTS Writers")
    
    #Create a table called Writers, with an Id column and a Name column
    cur.execute("CREATE TABLE Writers(Id INT PRIMARY KEY AUTO_INCREMENT, \
                 Name VARCHAR(25))")
    
    #Insert five entries to the Writers table.
    #Note that Id is auto-incremented from the previous line of code
    cur.execute("INSERT INTO Writers(Name) VALUES('Jack London')")
    cur.execute("INSERT INTO Writers(Name) VALUES('Honore de Balzac')")
    cur.execute("INSERT INTO Writers(Name) VALUES('Lion Feuchtwanger')")
    cur.execute("INSERT INTO Writers(Name) VALUES('Emile Zola')")
    cur.execute("INSERT INTO Writers(Name) VALUES('Truman Capote')")

<h1> Fetch data from the database (all at once, with default cursor) </h1>

In [4]:
con = mdb.connect('localhost', 'testuser', 'test623', 'testdb');

with con: 

    cur = con.cursor()
    
    #Send select statement
    cur.execute("SELECT * FROM Writers")

    #Get all of the MySQL return at once
    #Returns a tuple of tuples, with each inner tupple being one row
    rows = cur.fetchall()

    #Loop through the tuple of rows, and print the contents of each row
    for row in rows:
        print(row)

(1, 'Jack London')
(2, 'Honore de Balzac')
(3, 'Lion Feuchtwanger')
(4, 'Emile Zola')
(5, 'Truman Capote')


<h1> Fetch data from the database (one row at a time, with default cursor) </h1>

In [5]:
con = mdb.connect('localhost', 'testuser', 'test623', 'testdb');

with con:

    cur = con.cursor()
    cur.execute("SELECT * FROM Writers")

    #The .rowcount attribute keeps track of the number of rows 
    #returned by the execute statement
    for i in range(cur.rowcount):
        
        #Use .fetchone to read the rows one by one
        row = cur.fetchone()
        
        #Individual rows are returned as a list
        #Each list entry contains individual column entires for that row
        print ([column for column in row])

[1, 'Jack London']
[2, 'Honore de Balzac']
[3, 'Lion Feuchtwanger']
[4, 'Emile Zola']
[5, 'Truman Capote']


<h1> Fetch data from the database (all at once, with dict cursor) </h1>

In [6]:
con = mdb.connect('localhost', 'testuser', 'test623', 'testdb')

with con:
    
    #Specify the type of cursor with cursors.TYPE
    #The SQL return will now be a dict with column keys instead of a tuple
    cur = con.cursor(mdb.cursors.DictCursor)
    cur.execute("SELECT * FROM Writers LIMIT 4")

    rows = cur.fetchall()

    #Each row is a dict, with keys equal to the column names
    for row in rows:
        print (row["Id"], row["Name"])

1 Jack London
2 Honore de Balzac
3 Lion Feuchtwanger
4 Emile Zola


<h1> Get column names </h1>

In [7]:
con = mdb.connect('localhost', 'testuser', 'test623', 'testdb')

with con:

    cur = con.cursor()
    cur.execute("SELECT * FROM Writers LIMIT 5")

    rows = cur.fetchall()

    #Fetches the table description
    desc = cur.description

    #Desc is a list of lists, and the [:][0] entries are the column names
    print ("%s %3s" % (desc[0][0], desc[1][0]))

    for row in rows:    
        print ("%2s %3s" % row)

Id Name
 1 Jack London
 2 Honore de Balzac
 3 Lion Feuchtwanger
 4 Emile Zola
 5 Truman Capote


<h1> Querying the DB with a prepared statement </h1>

In [8]:
con = mdb.connect('localhost', 'testuser', 'test623', 'testdb')
    
with con:    

    cur = con.cursor()
        
    #This query updates the name of an author where the row Id =4
    #The statement uses ANSI printf format to replace %s placeholders
    cur.execute("UPDATE Writers SET Name = %s WHERE Id = %s", 
        ("Guy de Maupasant", "4"))        
    
    print ("Number of rows updated:",  cur.rowcount)

Number of rows updated: 1


<h1> Inserting images to the database </h1>

<h> Creating a new table to hold the images </h>

In [9]:
#Creating a new table named "Images" to hold the images

con = mdb.connect('localhost', 'testuser', 'test623', 'testdb');

with con:
    
    cur = con.cursor()
    
    #Remove the table if it already exists
    cur.execute("DROP TABLE IF EXISTS Images")

    #Create a table called Images, with an Id column and a Data column
    #The data colum is a "medium blob" type -- what SQL calls binary types
    cur.execute("CREATE TABLE Images(Id INT PRIMARY KEY,DATA MEDIUMBLOB);")
    

<h> Inserting the image </h>

In [10]:
con = mdb.connect('localhost', 'testuser', 'test623', 'testdb')
 
with con:
    #Get the binary data from the image
    #We use "blob_value" for the name because MySQL calls binary types blobs
    image_dir = '/Users/richardknoche/Downloads/PythonImg.jpg'
    blob_value = open(image_dir, 'rb').read()
    
    #Create cursor
    cursor=con.cursor()

    #Insert the image
    sql = 'INSERT INTO Images VALUES(1,%s)'  #the first VALUES(N,%s) is the row index
    args = (blob_value, )
    cursor.execute(sql,args)

<h1> Retrieving images from the database </h1>

In [11]:
con = mdb.connect('localhost', 'testuser', 'test623', 'testdb')

with con:

    #Create the cursor
    cur = con.cursor()
    
    #Query the DB for the image
    cur.execute("SELECT Data FROM Images WHERE Id=1")
    data = cur.fetchone()[0]
    
    #Write the image data to an output file
    out_path = '/Users/richardknoche/Downloads/PythonImg_FromDB.jpg'
    fout = open(out_path,'wb')
    with fout:
        fout.write(data)